# Baseline solution for seismic facies identification challenge

### Architecture: U-Net
### Language: Tensorflow/keras

In [1]:
import tensorflow as tf
import keras as K

from keras.layers import Input, BatchNormalization, Dropout, Dense, Activation
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

import numpy as np
from tqdm import tqdm
import cv2


import jaccard_loss as jc

### Define Network architecture

In [3]:
# defines convolutional block

def conv2d_block(input_tensor,n_filters, kernel_size=3, batchnorm=True):
    x = Conv2D(n_filters, (kernel_size,kernel_size),padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(n_filters, (kernel_size, kernel_size), padding='same',\
               kernel_initializer='he_normal')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

# defines unet architecture

def unet(img_input, out_channel, n_filters=64, dropout=0.1, k_size=3, batchnorm = True):
    c1 = conv2d_block(img_input, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = conv2d_block(p1, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    p2 = MaxPooling2D((2,2))(c2)
    
    c3 = conv2d_block(p2, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    p3 = MaxPooling2D((2,2))(c3)
    
    c4 = conv2d_block(p3, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    p4 = MaxPooling2D((2,2))(c4)
    
    c5 = conv2d_block(p4, n_filters*16, batchnorm=batchnorm)
    
    
    
    u6 = Conv2DTranspose(n_filters*8, (k_size,k_size), strides = (2,2), padding='same')(c5)
    u6 = concatenate([u6,c4])
    c6 = conv2d_block(u6, n_filters*8, kernel_size=k_size, batchnorm=batchnorm)
    
    u7 = Conv2DTranspose(n_filters*4, (k_size,k_size),strides = (2,2), padding='same')(c6)
    u7 = concatenate([u7,c3])
    c7 = conv2d_block(u7, n_filters*4, kernel_size=k_size, batchnorm=batchnorm)
    
    u8 = Conv2DTranspose(n_filters*2, (k_size,k_size), strides = (2,2), padding='same')(c7)
    u8 = concatenate([u8,c2])
    c8 = conv2d_block(u8, n_filters*2, kernel_size=k_size, batchnorm=batchnorm)
    
    u9 = Conv2DTranspose(n_filters*1, (k_size,k_size), strides = (2,2), padding='same')(c8)
    u9 = concatenate([u9,c1])
    c9 = conv2d_block(u9, n_filters*1, kernel_size=k_size, batchnorm=batchnorm)
    
    outputs = Conv2D(out_channel, (1,1), activation='softmax')(c9)
    model = Model(inputs= [img_input], outputs= [outputs], name ='U-Net')
    
    return model

## Data pre-processing

In [4]:
data_train = np.load('./dataset/data_train.npz')['data']
label_train = np.load('./dataset/labels_train.npz')['labels']


data_patch = np.concatenate((data_train[:,:,:295], data_train[:,:,295:]))
label_patch = np.concatenate((label_train[:,:,:295], label_train[:,:,295:]))

In [5]:
data_train.shape

(1006, 782, 590)

In [6]:
training_image_data = []
training_label_data = []

for i in tqdm (range(0, 2012)):
    image = data_patch[i,:,:]
    label = label_patch[i,:,:]
    
    image = np.expand_dims(image,axis=2).astype('float32')
    label = np.expand_dims(label,axis=2).astype('float32')
    
    image = cv2.resize(image, (256,256))
    label = cv2.resize(label,(256,256))
    
    training_image_data.append(image)
    training_label_data.append(label)
    
    
# convert data to array
training_image_data = np.asarray(training_image_data)
training_label_data = np.asarray(training_label_data)
training_label_data = np.array(training_label_data, dtype=int)

100%|██████████| 2012/2012 [00:11<00:00, 182.15it/s]


In [7]:
class DataGenerator():
    def __init__(self, x_set, y_set):
        self.x, self.y = x_set, y_set

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        batch_x = self.x[index]
        batch_y = self.y[index]
        #print(np.expand_dims(batch_x,axis=0), batch_y)
        return np.expand_dims(batch_x,axis=0), batch_y
    
train_dataset = DataGenerator(x_set=training_image_data,y_set=training_label_data) 

In [8]:
train_dataset.y

array([[[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       [[4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        ...,
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4],
        [4, 4, 4, ..., 4, 4, 4]],

       ...,

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 

In [12]:
# Define model
cropsize = 256
lr = 0.0001
n_classes = 7
y_true = Input(shape=(cropsize,cropsize,1), name='y_true')
model = unet(y_true, 7, n_filters=16, dropout=0.05, batchnorm=True)
model = Model(inputs=model.inputs, outputs=model.outputs)

# Optimizer
optim = K.optimizers.Adam(lr)

# Define loss 
#loss = generalized_dice_loss_w(y_true, model.get_layer("softmax").output)
#model.add_loss(loss)
jaccard_loss = jc.dice_coef_9cat_loss(y_true, model.output)

#iou = jc.iou_score(y_true, model.output, n_classes, class_weights=1.)[0]
#print('iou',iou)

#dice_loss = sm.losses.DiceLoss(class_weights=weights) 
model.add_loss(jaccard_loss)

#model.add_metric(iou, name=iou)

metrics = [tf.keras.metrics.MeanIoU(num_classes=7)]

model.compile(optimizer= optim, loss= None, metrics=metrics)

AttributeError: module 'tensorflow.keras.metrics' has no attribute 'IoU'

In [81]:
loss_history = []
acc_history = []
miou_history = []
num_epochs = 5
batch_size = 32
train_image = data
n_batches_train = len(train_image)//batch_size
n_batches_val = len(val_image)//batch_size


for epoch in range(num_epochs):
    print('Starting epoch {}-{}'.format(epoch+1, num_epochs))
    batch_start = 0 # Initial batch value of j epoch
    batch_end = batch_size  # Initial batch end of j epoch
    loss = 0
    iou_train = 0
    previous_loss = 99999 # initial previous_loss to put loss_val
    previous_loss_val = 99999
    
    start_time = time.time()
    for i in range(n_batches_train):
        history = model.train_on_batch([train_image[batch_start:batch_end],
                                        train_label[batch_start:batch_end]],
                                       None)
        #print(history)
        batch_start+=batch_size
        batch_end+=batch_size
        loss+=history[0]
        iou_train += history[1]
    end_time = time.time()
    print("epoch time: ",end_time - start_time)
    
    loss = loss/n_batches_train # Average loss function of all batches in j epoch (train)
    iou_train = iou_train/n_batches_train # Average IOU of all batches in j epoch (train)
    
    #Val
    
    batch_start_val = 0
    batch_end_val = batch_size
    loss_val = 0
    iou_val = 0
    
    for i in range(n_batches_val):  
        history_val = model.test_on_batch([val_image[batch_start_val:batch_end_val],
                                           val_label[batch_start_val:batch_end_val]],
                                         None)
        batch_start_val+=batch_size
        batch_end_val+=batch_size
        loss_val+=history_val[0]
        iou_val += history_val[1]
        
    loss_val = loss_val/n_batches_val
    iou_val = iou_val/n_batches_val
    
    if loss_val < previous_loss_val:
        model.save_weights('diceloss_unet_' + str(cropsize)+ "_" + backbone_net + "_nclass_"
                           + str(n_classes) + '_seg_' + 'weights.h5')
        previous_loss_val = loss_val
        
    if loss < previous_loss:
        model.save_weights('diceloss_unet_' + str(cropsize) + "_" + backbone_net + "_nclass_" 
                           + str(n_classes) + '_seg_' + 'weights_best_Train.h5')
        loss_ant = loss


    print("loss train: ",loss)
    print("iou train: ",iou_train)
    print("loss val: ",loss_val)

NameError: name 'train_image' is not defined

In [36]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
input_img = Input(x_size, name='img')
model = unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

In [89]:
train_dataset.y.shape

(2012, 256, 256)

In [83]:
dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [84]:
dataloader